# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
readpath = Path.cwd() / 'cities.csv'
df = pd.read_csv(readpath, index_col=0)
df.head()

,Long Guess,CityName,CountryCode,Temperature,Humidity,Cloudiness,Wind Speed,Lat Real,Long Real,Datetime
Lat Guess,,,,,,,,,,
59.933350,-7.833893,vagur,fo,48.69,86.0,100.0,21.59,61.47,-6.81,1.603849e+09
44.473937,25.474566,gratia,ro,57.00,93.0,90.0,4.70,44.43,25.45,1.603849e+09
-0.258064,-158.032534,faanui,pf,79.65,83.0,100.0,18.72,-16.48,-151.75,1.603849e+09
-28.117164,-105.514850,rikitea,pf,75.52,83.0,100.0,15.48,-23.12,-134.97,1.603849e+09
-18.473594,-178.352367,vaini,to,77.00,78.0,75.0,16.11,-21.20,-175.20,1.603849e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
Humiditylocations = df[['Lat Real', 'Long Real']]
weights = df['Humidity']*10000
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(Humiditylocations, weights=weights, point_radius=20))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    - 65-85 F
    - Wind Speed <15
    - 50% Cloudiness
* Drop any rows will null values.

In [5]:
# Filter a COPY of the original dataframe based on above parameters for a "perfect day"
df_copy = df.copy()
df_copy = df_copy.loc[df_copy['Temperature'] >= 70 ]
df_copy = df_copy.loc[df_copy['Temperature'] <= 85 ]
df_copy = df_copy.loc[df_copy['Wind Speed'] <= 10]
df_copy = df_copy.loc[df_copy['Cloudiness'] <= 50]
print(f'The remaining number of cities that fit our criteria is {len(df_copy)}')

The remaining number of cities that fit our criteria is 71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Set up our basic query
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "hotel",
    "distance": 5000,
    "rankby": "distance",
    "key": g_key
}

In [7]:
df_copy.reset_index(inplace=True)
df_copy.head()

,Lat Guess,Long Guess,CityName,CountryCode,Temperature,Humidity,Cloudiness,Wind Speed,Lat Real,Long Real,Datetime
0,-47.275774,81.601122,mahebourg,mu,70.00,80.0,8.0,3.00,-20.41,57.70,1.603849e+09
1,9.947242,-19.185517,bubaque,gw,82.15,80.0,21.0,9.64,11.28,-15.83,1.603849e+09
2,-25.739976,38.144261,inhambane,mz,71.67,90.0,2.0,7.36,-23.86,35.38,1.603849e+09
3,-23.358221,136.954159,alice springs,au,82.40,12.0,0.0,5.82,-23.70,133.88,1.603849e+09
4,-4.092311,121.163857,bone,id,83.91,53.0,19.0,6.17,-9.83,124.45,1.603849e+09


In [8]:
# Store into variable named hotel_df
hotel_df = df_copy
hotel_df['Hotel Name'] = ''
for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Lat Real']},{row['Long Real']}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest lodging is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan # Added this to make it possible to use .dropna()

Closest lodging is Garden Lake Villa.
Closest lodging is Apart Hotel New Neguen.
Closest lodging is Hotel Casa Do Capitão.
Closest lodging is Elkira Court Motel.
Closest lodging is Dena Hotel.
Closest lodging is Horisun.
Closest lodging is Best Western Plus Sanford Airport/Lake Mary Hotel.
Closest lodging is OYO 44821 Hotel Paane.
Missing field/result... skipping.
Closest lodging is Villa Pacheco.
Closest lodging is Pousada Fulô de Maringá.
Closest lodging is Garden Lake Villa.
Closest lodging is Hotel Casa Do Capitão.
Closest lodging is The St George's Club & Resort.
Missing field/result... skipping.
Closest lodging is Auberge De Souillac.
Closest lodging is O Carinhanha Hotel.
Closest lodging is The St George's Club & Resort.
Closest lodging is Hilltop Boutique Hotel.
Closest lodging is REER XOOSHLE GROUP.
Closest lodging is BEN-HUR Hotel.
Closest lodging is Garden Lake Villa.
Closest lodging is Hilltop Boutique Hotel.
Closest lodging is Garden Lake Villa.
Closest lodging is My Indra

In [9]:
hotel_df.rename(columns={"CityName": "City", "CountryCode": "Country", "Lat Real": "Lat", "Long Real": "Lng"}, inplace=True)
hotel_df.dropna(inplace=True)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))